In [1]:
!pip install -q py-feat
# !pip install -q facial-emotion-recognition
# !pip install -q fer
# !pip install -q deepface
# !pip install -q mtcnn

     |████████████████████████████████| 6.0MB 8.6MB/s 
     |████████████████████████████████| 3.3MB 34.8MB/s 
     |████████████████████████████████| 57.4MB 46kB/s 
     |████████████████████████████████| 4.9MB 34.1MB/s 


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import numpy as np
import pandas as pd
from feat import Detector
import glob
import math
import dlib
import joblib
import cv2
from tqdm.notebook import tqdm
from google.colab.patches import cv2_imshow
from collections import Counter

import tensorflow as tf
from keras.applications.vgg19 import VGG19, preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.layers import GlobalAveragePooling2D, Dense, Dropout, Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import MaxPooling2D
from keras.models import Model
from keras import optimizers 
from keras import regularizers
from keras.callbacks import ModelCheckpoint,EarlyStopping,ReduceLROnPlateau

face_model = "RetinaFace"
landmark_model = "MobileNet"
au_model = "svm"
emotion_model = "resmasknet" #resmasknet,fer, svm, rf
detector = Detector(face_model = face_model, landmark_model = landmark_model, au_model = au_model, emotion_model = emotion_model)

def my_mode(sample):
  c = Counter(sample)
  return [k for k, v in c.items() if v == c.most_common(1)[0][1]]

Mounted at /content/drive


## ALL STEP IN ONE

In [ ]:
base_model = VGG19(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

for layer in base_model.layers:
    layer.trainable = False

x = base_model.output
x = Flatten()(x)
x = Dense(1024)(x)
x = Dropout(0.5)(x)
x = Dense(512)(x)
x = Dropout(0.5)(x)
output = Dense(4, activation='softmax')(x)

# Create  model
model_4 = Model(inputs=base_model.input, outputs=output)

base_model_layers = model_4.layers[:23]
new_model_layers =  model_4.layers[23:]

base_model_blocks = {
    0: base_model_layers[1:3],
    1: base_model_layers[4:6],
    2: base_model_layers[7:11],
    3: base_model_layers[12:16],
    4: base_model_layers[17:21]
}

# Load the best model
model_4_path = "./drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/model_ck_kaggle_jaffe/{}.h5".format("model_4")
model_4.load_weights(model_4_path)

In [ ]:
ls 'drive/MyDrive/file2/vdo/Isaree/01_karnmay.mp4'

In [ ]:
ls 'drive/MyDrive/file2/done/'

In [ ]:
# ls 'drive/MyDrive/AIHealthcare/AIcare_Phrase1/data/model_FERG_DB_256'

In [ ]:
pred_emotion= []

filepath = 'drive/MyDrive/file2/done/face2_01_KARNMAY_P1.csv'
vdopath ='drive/MyDrive/file2/vdo/Isaree/01_karnmay.mp4'
df = pd.read_csv(filepath)
dim = (224, 224)
rowimg = []
label_img = []
print('df', df.shape)

for i in tqdm(range(df.shape[0])):
  seqimg = []
  for j in range(df['start_frame'][i], df['end_frame'][i], 3):
    cap = cv2.VideoCapture(vdopath)
    cap.set(cv2.CAP_PROP_POS_FRAMES, j)
    success, image = cap.read()
    faceimg = detector.detect_faces(image)
    try:
      pos = faceimg[0][:4]
      cropimg =image[int(pos[1]):int(pos[3]),int(pos[0]):int(pos[2])] 
      resized = cv2.resize(cropimg, dim, interpolation=cv2.INTER_CUBIC)     
      image = preprocess_input(resized)
      image = np.expand_dims(image, axis=0)
      preds = model_4.predict(image)
      pred_class = np.argmax(preds)
      seqimg.append(pred_class) 
    except:
      print("Emotion Error  i = ",i, ' j =', j)

  seq_label = my_mode(seqimg)
  print('i =', i, 'seq_label =', seq_label)
  try:
    pred_emotion.append(seq_label[0])
  except:
    pass

In [ ]:
pred_emotion= []

filepath = 'drive/MyDrive/file2/done/face2_01_KARNMAY_P1.csv'
vdopath ='drive/MyDrive/file2/vdo/Isaree/01_karnmay.mp4'
df = pd.read_csv(filepath)
dim = (224, 224)
rowimg = []
label_img = []
print('df', df.shape)

for i in tqdm(range(86, df.shape[0])):
  seqimg = []
  for j in range(df['start_frame'][i], df['end_frame'][i], 3):
    cap = cv2.VideoCapture(vdopath)
    cap.set(cv2.CAP_PROP_POS_FRAMES, j)
    success, image = cap.read()
    faceimg = detector.detect_faces(image)
    try:
      pos = faceimg[0][:4]
      cropimg =image[int(pos[1]):int(pos[3]),int(pos[0]):int(pos[2])] 
      resized = cv2.resize(cropimg, dim, interpolation=cv2.INTER_CUBIC)     
      image = preprocess_input(resized)
      image = np.expand_dims(image, axis=0)
      preds = model_4.predict(image)
      pred_class = np.argmax(preds)
      seqimg.append(pred_class) 
    except:
      print("Emotion Error  i = ",i, ' j =', j)

  seq_label = my_mode(seqimg)
  print('i =', i, 'seq_label =', seq_label)
  try:
    pred_emotion.append(seq_label[0])
  except:
    pass

df (152, 8)


i = 86 seq_label = [2]
i = 87 seq_label = [2]
i = 88 seq_label = [2]
i = 89 seq_label = [2]
i = 90 seq_label = [2]
i = 91 seq_label = [2]
i = 92 seq_label = [2]
i = 93 seq_label = [2]
i = 94 seq_label = [2]
Emotion Error  i =  95  j = 51860
Emotion Error  i =  95  j = 51863
Emotion Error  i =  95  j = 51866
Emotion Error  i =  95  j = 51869
Emotion Error  i =  95  j = 51872
Emotion Error  i =  95  j = 51875
Emotion Error  i =  95  j = 51878
Emotion Error  i =  95  j = 51881
Emotion Error  i =  95  j = 51884
Emotion Error  i =  95  j = 51887
Emotion Error  i =  95  j = 51890
Emotion Error  i =  95  j = 51893
Emotion Error  i =  95  j = 51896
Emotion Error  i =  95  j = 51899
Emotion Error  i =  95  j = 51902
Emotion Error  i =  95  j = 51905
Emotion Error  i =  95  j = 51908
Emotion Error  i =  95  j = 51911
Emotion Error  i =  95  j = 51914
Emotion Error  i =  95  j = 51917
Emotion Error  i =  95  j = 51920
Emotion Error  i =  95  j = 51923
Emotion Error  i =  95  j = 51926
Emotion Err

In [ ]:
len(pred_emotion)

65

In [ ]:
df = pd.DataFrame(np.array(pred_emotion), columns=['label'])
from google.colab import files
df.to_csv('face2_01_KARNMAY_model_ck_kaggle_jaffe.csv') 
files.download('face2_01_KARNMAY_model_ck_kaggle_jaffe.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
while True:pass

KeyboardInterrupt: ignored

In [ ]:
dic_resmask =  {0:'anger',1:'disgust', 2:'fear', 3:'happiness',4:'sadness',	5:'surprise',	6:'neutral' }
# dic_kagle =  {0:'anger',1:'disgust', 2:'fear', 3:'happiness',4:'sadness',	5:'surprise',	6:'neutral' }
# dic_ckplus =  {1:'anger',3:'disgust', 4:'fear', 5:'happiness',6:'sadness',	7:'surprise'}
# dic_fer_svm_rf =  {0:'anger',1:'disgust', 2:'fear', 3:'happiness',4:'sadness',	5:'surprise',	6:'neutral' }
# dic_fer_svm_rf =  {0:'anger',1:'disgust', 2:'fear', 3:'happiness',4:'sadness',	5:'surprise',	6:'neutral' }
# dic_Deepfake =  {0:'anger',1:'fear', 2:'neutral', 3:'sad',4:'disgust',	5:'happy',	6:'surprise' }

pred = pd.DataFrame({'Pred':pred_emotion})
pred['emotion'] = pred['Pred'].replace(dic_resmask) 
result = pd.concat([df, pred], axis=1, join="inner")

from google.colab import files
result.to_csv('05_pimrypie_tony_R1.csv') 
files.download('05_pimrypie_tony_R1.csv')